In [51]:


import os
import pandas as pd
import math
import tarfile
from six.moves import urllib
import hashlib
import numpy as np
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tokenize import TreebankWordTokenizer
from bs4 import BeautifulSoup
import xml.etree.ElementTree
import mailbox   
import lxml.html
import lxml.html.clean
from sklearn.base import BaseEstimator, TransformerMixin
from nltk.stem import PorterStemmer
import email
from sklearn.pipeline import Pipeline
# Import the email modules we'll need
from email.parser import Parser
import mailparser as mp
from email.mime.text import MIMEText
import nltk
from nltk import pos_tag
import re
from collections import Counter



# path = spam_path
# i = 0
# for filename in os.listdir(path):
#     print('orig', filename)
#     os.rename(os.path.join(path,filename), os.path.join(path,'mail_'+str(i)))
#     print('changed', filename)
#     i = i +1
# print(i)




class preparing_data(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    def fit(self, x, y=None):
        return self
    def transform(self, x, y=None):
        #print('type of x is' , type(x))
        #x = x.lower()
        m_body, m_headers, sub = dividing_mail(x)
        if m_body != '':
            m_body = m_body.lower()
            #m_body = m_body.split('.').join("");
            urls = counting_urls(m_body)
            m_body = cleaning_htmlent(m_body)
            
            name_sender, dom_sender = x.from_[0][0], x.from_[0][1].split('@')[-1]
        
            sub_list = keep_only_important_words(sub)
            body_list = keep_only_important_words(m_body)
            sub_list = removing_alphas(sub_list)
            body_list = removing_alphas(body_list)
            
            #save words record?
            ps = PorterStemmer()
            s_c =   [ps.stem(w) for w in sub_list]
            b_c =   [ps.stem(w) for w in body_list]
            b_c.extend([urls, name_sender, dom_sender])
            
            
        else:
            print('returning nothing')
            s_c =['','','']
            b_c = ['0','','']
        return s_c, b_c
        



            
def keep_only_important_words(data_txt):
        m_words =[]
        for word, pos in nltk.pos_tag(TreebankWordTokenizer().tokenize(data_txt)):
            #if pos not in ['IN', '.', 'CD', 'DT', 'RB', 'VBP', 'TO', 'PRP', 'C']: VB and VBP removed
            if pos in ['NN',  'JJ', 'VBN'] :
                m_words.append(word)
            else:
                pass
        return(m_words)

def cleaning_htmlent(text):
    if text != '':
        doc = lxml.html.fromstring(text)
        cleaner = lxml.html.clean.Cleaner(style=True)
        doc = cleaner.clean_html(doc)
        content = doc.text_content()
        return content
    else:
        return []

def counting_urls(text):
#     soup = BeautifulSoup(text, 'html.parser')
#     num = soup.find_all('a')
#     print('number is ', (num))

    urls = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', text)
    num_urls = len(urls)
    return num_urls
  
def dividing_mail(mail):
        m_body = mail.body
        m_headers = mail.headers
        
        sub = mail.subject
        return m_body, m_headers, sub

def removing_alphas(doc):
    dellist = []
    for i in range(len(doc)):
        if not doc[i].isalpha():
            dellist.append(i)
       
    for index in sorted(dellist, reverse=True):
        del doc[index]
    return doc
    
    

def extracting_feats(body_list, sub_list, arr):
    i = 0
    fl = []
    for  l in list([body_list, sub_list]):
        flatten_list = [a for t in l for a in t]
        out = Counter(flatten_list)
        out = out.most_common(arr[i])
        i = i + 1 
        cols = [m[0] for m in out]
        fl.append(cols)
    return fl
      
    
    
    
sub_list = []
body_list = []




#Giving filepaths for the data
dat_path = "datasets/spamham"
spam_path = dat_path + '/easy_spam'
ham_path = dat_path + '/easy_ham'
len_spam = len(next(os.walk(spam_path))[2])
len_ham = len(next(os.walk(ham_path))[2])
ar_dict = [(spam_path, len_spam), (ham_path, len_ham)] 
sl = math.ceil(0.8 * len_spam)
hl = math.ceil(0.8 * len_ham)

y_train = [0] * sl + [1] * hl
n_list = []
    
for sp, num in ar_dict:
    tr_n = math.ceil(0.8 * int(num))
    print('trainining number: ', tr_n)
    
    for file_num in range(tr_n):
        #Reading an email and getting the content
        data_mail = sp + '/mail_' + str(file_num)
        #print(data_mail)
        
        
        mail = mp.parse_from_file(data_mail)
        obj = preparing_data()
        s, b = obj.fit_transform(mail)
        n_list.append(b[-3:])
        sub_list.append(s)
        body_list.append(b[:-3])
        
            
featurevectors = extracting_feats(body_list[:sl], sub_list[:sl], [200 ,50])
featurevectorh = extracting_feats(body_list[sl:], sub_list[sl:], [300 ,50])
#feat_vals = set(featurevectors) + set(featurevectorh)
print((body_list[679]))


trainining number:  400
returning nothing
returning nothing
trainining number:  2041


More than one match found for (?:with(?! cipher)\s+(?P<with>.+?)(?:\s*[(]?envelope-from|\s*[(]?envelope-sender|\s+from|\s+by|\s+id|\s+for|\s+via|;)) in from unknown HELO ?192.168.0.100? salimma1@212.18.241.211 with plain by smtp.mail.vip.sc5.yahoo.com with SMTP; 10 Oct 2002 10:30:25 -0000
More than one match found for (?:with(?! cipher)\s+(?P<with>.+?)(?:\s*[(]?envelope-from|\s*[(]?envelope-sender|\s+from|\s+by|\s+id|\s+for|\s+via|;)) in from p977.as2.cra.dublin.eircom.net HELO mfrenchw2k mfrench42@159.134.179.209 with login by smtp.mail.vip.sc5.yahoo.com with SMTP; 22 Aug 2002 22:02:25 -0000
More than one match found for (?:id\s+(?P<id>.+?)(?:\s*[(]?envelope-from|\s*[(]?envelope-sender|\s+from|\s+by|\s+with(?! cipher)|\s+for|\s+via|;)) in from chassid 4.65.20.230 by out001.verizon.net InterMail vM.5.01.05.09 201-253-122-126-109-20020611 with ESMTP id <20021005011206.OGMC3265.out001.verizon.net@chassid> for <rpm-list@freshrpms.net>; Fri, 4 Oct 2002 20:12:06 -0500
More than one match fo

['use', 'perl', 'daili', 'headlin', 'mailer', 'parrot', 'pumpkin', 'post', 'km', 'tuesday', 'parrot', 'http', 'perl', 'journal', 'post', 'pudg', 'tuesday', 'http', 'copyright', 'receiv', 'messag', 'use', 'receiv', 'other', 'use', 'perl', 'pleas', 'user', 'http']


In [53]:
bodycols = set(featurevectors[0] + featurevectorh[0])
subcols = set(featurevectors[1] + featurevectorh[1])
print(bodycols)

sample_value = []


for num in range(hl + sl):
    for feature in bodycols:
        #print(feature)
        if feature in body_list[num]:
            sample_value.append(body_list[num].count(feature))
            
        else:
            sample_value.append(0)
                #print(sample_value)
    for feature in subcols:
        #print(len(sample_value))
        if feature in sub_list[num]:
            sample_value.append(sub_list[num].count(feature))
                #print(sample_value)
        else:
            sample_value.append(0)
            #print(sample_value)
    urlno = n_list[num][0]
    sample_value.append(urlno)
            
   
print(len(sample_value))

{'possibl', 'profit', 'paid', 'applic', 'run', 'copi', 'test', 'exampl', 'un', 'link', 'one', 'nation', 'build', 'copyright', 'addit', 'hardwar', 'onli', 'bitbitch', 'new', 'script', 'done', 'error', 'great', 'msg', 'post', 'stori', 'easi', 'good', 'simpl', 'high', 'cost', 'type', 'java', 'order', 'access', 'folder', 'system', 'point', 'mailto', 'process', 'design', 'mortgag', 'listmast', 'year', 'own', 'thank', 'street', 'old', 'bug', 'open', 'mon', 'subject', 'sort', 'info', 'commun', 'report', 'man', 'nice', 'drive', 'softwar', 'secur', 'domain', 'network', 'visit', 'spam', 'human', 'kind', 'larg', 'exmh', 'whole', 'industri', 'center', 'servic', 'alb', 'next', 'first', 'gnupg', 'direct', 'kingdom', 'chang', 'septemb', 'monday', 'opportun', 'version', 'click', 'someth', 'membership', 'financi', 'packag', 'sep', 'such', 'discuss', 're', 'major', 'unit', 'day', 'job', 'freedom', 'welcom', 'enenkio', 'c', 'interest', 'thu', 'number', 'window', 'polit', 'limit', 'perl', 'suppli', 'forc'

In [58]:

colnames =  list(bodycols) + list(subcols) + ['URL_num']
numtoreshape = len(colnames)
data = np.reshape(sample_value, (2441, numtoreshape)) 
#print(data.shape)
#print(colnames)

df = pd.DataFrame(data = data, columns = colnames)
print(df.loc[1]['incom'])

1


In [33]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
clf = RandomForestClassifier(n_estimators=100, random_state=42) 

result = cross_val_score(clf, df.values, y_train, cv = 3, scoring = "accuracy")
print(result)

[0.99263804 0.98277983 0.97662977]


In [9]:
body_list[-20:]

[['partit',
  'particular',
  'scheme',
  'i',
  'i',
  'reshuffl',
  'case',
  'fwiw',
  'i',
  'mh',
  'style',
  'number',
  'mh',
  'freak',
  'sort',
  'train',
  'ham',
  'spam',
  'train',
  'ham',
  'spam',
  'train',
  'ham',
  'spam',
  'train',
  'ham',
  'spam',
  'train',
  'ham',
  'spam',
  'total',
  'fals',
  'po',
  'total',
  'fals',
  'neg',
  'shuffl',
  'i',
  'total',
  'fals',
  'po',
  'total',
  'fals',
  'neg',
  'sake',
  'comparis',
  'partit',
  'train',
  'ham',
  'spam',
  'train',
  'ham',
  'spam',
  'total',
  'fals',
  'po',
  'total',
  'fals',
  'neg',
  'later',
  'anthoni'],
 ['url',
  'http',
  'date',
  'disput',
  'widow',
  'kurt',
  'cobain',
  'band',
  'nirvana',
  'been',
  'settl',
  'way',
  'new',
  'cd'],
 ['friend',
  'dublin',
  'cd',
  'uk',
  'sunday',
  'i',
  'been',
  'advis',
  'machin',
  'danger',
  'http',
  'whump',
  'bill'],
 ['updat',
  'directori',
  'tag',
  'log',
  'messag',
  'index',
  'rc',
  'file',
  'revis',
 

In [18]:
def cleaning_htmlent(text):
    if text != '':
        doc = lxml.html.fromstring(text)
        cleaner = lxml.html.clean.Cleaner(style=True)
        doc = cleaner.clean_html(doc)
        content = doc.text_content()
        return content
    else:
        return []

em = "Content-Transfer-Encoding: quoted-printable<HTML><HEAD>gdaf</HEAD><BODY><FONT SIZE=3D 4 ><B> A man endowed with a 7 - 8 hammer is simply<br> better equipped than a man with a 5 - 6 hammer. <BR> <BR> ase your size by 1 - 3 <br> <A HREF=3D http://202.109.114.136/index.php?10= 004 >Enter here and see how</A></BODY></HTML>"

cleaned = cleaning_htmlent(em)
cleaned

'Content-Transfer-Encoding: quoted-printablegdaf A man endowed with a 7 - 8 hammer is simply better equipped than a man with a 5 - 6 hammer.   ase your size by 1 - 3  Enter here and see how'

In [60]:
df.shape[1]

485